# Online Course Recommendation System

## Importing Libraries

In [1]:
#!pip install tflearn==0.3.2

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
#import tflearn
from collections import Counter
import matplotlib.pyplot as plt
import random
import os
import pickle
import nltk
import json
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from scipy.sparse import hstack
%matplotlib inline

[nltk_data] Downloading package punkt to C:\Users\SANTOSH
[nltk_data]     ABHINAV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SANTOSH
[nltk_data]     ABHINAV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SANTOSH
[nltk_data]     ABHINAV\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Loading Dataset

In [3]:
df_edx = pd.read_csv("edX.csv")
df_edx.head()

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
0,How to Learn Online,Learn essential strategies for successful onli...,"1,24,980",Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2–3 hours per week,2 Weeks,FREE-Add a Verified Certificate for $49 USD,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...,https://www.edx.org/course/how-to-learn-online
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...","2,93,864",Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,2–4 hours per week,7 Weeks,FREE-Add a Verified Certificate for $49 USD,This course aims to teach everyone the basics ...,NaN,https://www.edx.org/course/programming-for-eve...
2,CS50's Introduction to Computer Science,An introduction to the intellectual enterprise...,"24,42,271",Self-paced on your time,Harvard University,David J. Malan-Doug Lloyd-Brian Yu,Introductory,Computer Science,English,English,6–18 hours per week,12 Weeks,FREE-Add a Verified Certificate for $90 USD,"This is CS50x , Harvard University's introduct...",NaN,https://www.edx.org/course/cs50s-introduction-...
3,The Analytics Edge,"Through inspiring examples and stories, discov...","1,29,555",Instructor-led on a course schedule,Massachusetts Institute of Technology,Dimitris Bertsimas-Allison O'Hair-John Silberh...,Intermediate,Data Analysis & Statistics,English,English,10–15 hours per week,13 Weeks,FREE-Add a Verified Certificate for $199 USD,"In the last decade, the amount of data availab...",NaN,https://www.edx.org/course/the-analytics-edge
4,Marketing Analytics: Marketing Measurement Str...,This course is part of a MicroMasters® Program,"81,140",Self-paced on your time,"University of California, Berkeley",Stephan Sorger,Introductory,Computer Science,English,English,5–7 hours per week,4 Weeks,FREE-Add a Verified Certificate for $249 USD,Begin your journey in a new career in marketin...,NaN,https://www.edx.org/course/marketing-analytics...


In [4]:
df_edx.columns

Index(['title', 'summary', 'n_enrolled', 'course_type', 'institution',
       'instructors', 'Level', 'subject', 'language', 'subtitles',
       'course_effort', 'course_length', 'price', 'course_description',
       'course_syllabus', 'course_url'],
      dtype='object')

## Preprocessing edX Dataset

In [5]:
df_edx.describe()

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
count,975,922,855,975,975,972,975,975,975,972,975,975,975,935,417,975
unique,972,887,849,2,111,775,3,31,9,33,53,18,47,932,412,974
top,Marketing Management,This course is part of a MicroMasters® Program,"97,291",Self-paced on your time,Harvard University,Rafael Irizarry,Introductory,Computer Science,English,English,2–4 hours per week,4 Weeks,FREE-Add a Verified Certificate for $49 USD,"About the Database Series of Courses""Databases...",None,https://www.edx.org/course/introduction-to-pro...
freq,2,15,2,917,104,9,621,166,777,713,108,194,270,3,3,2


### Feature Selection

In [6]:
df_edx = df_edx[["title", "institution", "Level", "n_enrolled"]]

In [7]:
df_edx.head()

,title,institution,Level,n_enrolled
0,How to Learn Online,edX,Introductory,"1,24,980"
1,Programming for Everybody (Getting Started wit...,The University of Michigan,Introductory,"2,93,864"
2,CS50's Introduction to Computer Science,Harvard University,Introductory,"24,42,271"
3,The Analytics Edge,Massachusetts Institute of Technology,Intermediate,"1,29,555"
4,Marketing Analytics: Marketing Measurement Str...,"University of California, Berkeley",Introductory,"81,140"


In [8]:
df_edx['title'].describe()
count = df_edx['title'].count()
count

975

### Cleaning the unwanted data

In [9]:
lst = []
for index in range(count):
    text = df_edx['title'].iloc[index]
    print("Text : ", text)
    print(wordpunct_tokenize(text))
    #Removing punctuations, numbers, converting into lower case
    filtered_text = " ".join(j.lower() for j in wordpunct_tokenize(text) if j.lower() not in stopwords.words('english'))
    filtered_text = "".join(i for i in filtered_text if i.isalpha() or i == " ")
    print("Filtered Text : ", filtered_text, "\n")
    df_edx['title'].iloc[index] = filtered_text
    lst.extend(filtered_text.split())   # Finding commonly occured words

Text :  How to Learn Online
['How', 'to', 'Learn', 'Online']
Filtered Text :  learn online 

Text :  Programming for Everybody (Getting Started with Python)
['Programming', 'for', 'Everybody', '(', 'Getting', 'Started', 'with', 'Python', ')']
Filtered Text :  programming everybody  getting started python  

Text :  CS50's Introduction to Computer Science
['CS50', "'", 's', 'Introduction', 'to', 'Computer', 'Science']
Filtered Text :  cs  introduction computer science 

Text :  The Analytics Edge
['The', 'Analytics', 'Edge']
Filtered Text :  analytics edge 

Text :  Marketing Analytics: Marketing Measurement Strategy
['Marketing', 'Analytics', ':', 'Marketing', 'Measurement', 'Strategy']
Filtered Text :  marketing analytics  marketing measurement strategy 

Text :  Introduction to Project Management
['Introduction', 'to', 'Project', 'Management']
Filtered Text :  introduction project management 

Text :  Leading High-Performing Teams
['Leading', 'High', '-', 'Performing', 'Teams']
Filte

Filtered Text :  accounting finance 

Text :  Computing in Python I: Fundamentals and Procedural Programming
['Computing', 'in', 'Python', 'I', ':', 'Fundamentals', 'and', 'Procedural', 'Programming']
Filtered Text :  computing python  fundamentals procedural programming 

Text :  Nutrition, Exercise and Sports
['Nutrition', ',', 'Exercise', 'and', 'Sports']
Filtered Text :  nutrition  exercise sports 

Text :  Becoming an Entrepreneur
['Becoming', 'an', 'Entrepreneur']
Filtered Text :  becoming entrepreneur 

Text :  Religion, Conflict and Peace
['Religion', ',', 'Conflict', 'and', 'Peace']
Filtered Text :  religion  conflict peace 

Text :  Tangible Things: Discovering History Through Artworks, Artifacts, Scientific Specimens, and the Stuff Around You
['Tangible', 'Things', ':', 'Discovering', 'History', 'Through', 'Artworks', ',', 'Artifacts', ',', 'Scientific', 'Specimens', ',', 'and', 'the', 'Stuff', 'Around', 'You']
Filtered Text :  tangible things  discovering history artworks  

Text :  Python: aprender a programar
['Python', ':', 'aprender', 'a', 'programar']
Filtered Text :  python  aprender programar 

Text :  Marketing Analytics: Competitive Analysis and Market Segmentation
['Marketing', 'Analytics', ':', 'Competitive', 'Analysis', 'and', 'Market', 'Segmentation']
Filtered Text :  marketing analytics  competitive analysis market segmentation 

Text :  MalariaX: Defeating Malaria from the Genes to the Globe
['MalariaX', ':', 'Defeating', 'Malaria', 'from', 'the', 'Genes', 'to', 'the', 'Globe']
Filtered Text :  malariax  defeating malaria genes globe 

Text :  Entrepreneurship in Emerging Economies
['Entrepreneurship', 'in', 'Emerging', 'Economies']
Filtered Text :  entrepreneurship emerging economies 

Text :  JavaScript Introduction
['JavaScript', 'Introduction']
Filtered Text :  javascript introduction 

Text :  People Management
['People', 'Management']
Filtered Text :  people management 

Text :  Using Email for Networking in English
['Using', 'Email', 

Filtered Text :  contemporary china  people  republic  taiwan  hong kong 

Text :  Introduction to Clinical Psychology
['Introduction', 'to', 'Clinical', 'Psychology']
Filtered Text :  introduction clinical psychology 

Text :  Modern Masterpieces of World Literature
['Modern', 'Masterpieces', 'of', 'World', 'Literature']
Filtered Text :  modern masterpieces world literature 

Text :  Citizen Politics in America: Public Opinion, Elections, Interest Groups, and the Media
['Citizen', 'Politics', 'in', 'America', ':', 'Public', 'Opinion', ',', 'Elections', ',', 'Interest', 'Groups', ',', 'and', 'the', 'Media']
Filtered Text :  citizen politics america  public opinion  elections  interest groups  media 

Text :  Applied Scrum for Agile Project Management
['Applied', 'Scrum', 'for', 'Agile', 'Project', 'Management']
Filtered Text :  applied scrum agile project management 

Text :  Corporate Finance
['Corporate', 'Finance']
Filtered Text :  corporate finance 

Text :  Judaism Through Its Scr

Filtered Text :  framing communication inspire convince 

Text :  Business Foundations
['Business', 'Foundations']
Filtered Text :  business foundations 

Text :  The Chemistry of Life
['The', 'Chemistry', 'of', 'Life']
Filtered Text :  chemistry life 

Text :  Nutrition and Health: Micronutrients and Malnutrition
['Nutrition', 'and', 'Health', ':', 'Micronutrients', 'and', 'Malnutrition']
Filtered Text :  nutrition health  micronutrients malnutrition 

Text :  Digital Design
['Digital', 'Design']
Filtered Text :  digital design 

Text :  Machine Learning for Data Science and Analytics
['Machine', 'Learning', 'for', 'Data', 'Science', 'and', 'Analytics']
Filtered Text :  machine learning data science analytics 

Text :  The Science of Everyday Thinking
['The', 'Science', 'of', 'Everyday', 'Thinking']
Filtered Text :  science everyday thinking 

Text :  Gestion de projets de développement
['Gestion', 'de', 'projets', 'de', 'développement']
Filtered Text :  gestion de projets de développ

...

Filtered Text :  probability statistics  gentle introduction probability 

Text :  Introducción a las ecuaciones diferenciales
['Introducción', 'a', 'las', 'ecuaciones', 'diferenciales']
Filtered Text :  introducción las ecuaciones diferenciales 

Text :  Agile Mindset Introduction
['Agile', 'Mindset', 'Introduction']
Filtered Text :  agile mindset introduction 

Text :  Tropical coastal ecosystems
['Tropical', 'coastal', 'ecosystems']
Filtered Text :  tropical coastal ecosystems 

Text :  Modern China’s Foundations: The Manchus and the Qing
['Modern', 'China', '’', 's', 'Foundations', ':', 'The', 'Manchus', 'and', 'the', 'Qing']
Filtered Text :  modern china  foundations  manchus qing 

Text :  El método de ciencia de datos
['El', 'método', 'de', 'ciencia', 'de', 'datos']
Filtered Text :  el método de ciencia de datos 

Text :  AWS Developer: Deploying on AWS
['AWS', 'Developer', ':', 'Deploying', 'on', 'AWS']
Filtered Text :  aws developer  deploying aws 

Text :  Autonomous Mobile R

Filtered Text :  modern japanese architecture part   meiji restoration pacific war 

Text :  Diagramas UML estructurales para la Ingeniería del Software
['Diagramas', 'UML', 'estructurales', 'para', 'la', 'Ingeniería', 'del', 'Software']
Filtered Text :  diagramas uml estructurales para la ingeniería del software 

Text :  A-level Mathematics for Year 12 - Course 2: Calculus, Newton’s Laws and Hypothesis Testing
['A', '-', 'level', 'Mathematics', 'for', 'Year', '12', '-', 'Course', '2', ':', 'Calculus', ',', 'Newton', '’', 's', 'Laws', 'and', 'Hypothesis', 'Testing']
Filtered Text :   level mathematics year   course   calculus  newton  laws hypothesis testing 

Text :  Fundamentos de Mecánica para Ingeniería
['Fundamentos', 'de', 'Mecánica', 'para', 'Ingeniería']
Filtered Text :  fundamentos de mecánica para ingeniería 

Text :  Leadership and Management for PM Practitioners in IT
['Leadership', 'and', 'Management', 'for', 'PM', 'Practitioners', 'in', 'IT']
Filtered Text :  leadership 

Filtered Text :  design healthcare  using patient journey mapping 

Text :  El manejo de plagas, malezas y enfermedades en los Sistemas de Siembra Directa
['El', 'manejo', 'de', 'plagas', ',', 'malezas', 'y', 'enfermedades', 'en', 'los', 'Sistemas', 'de', 'Siembra', 'Directa']
Filtered Text :  el manejo de plagas  malezas enfermedades en los sistemas de siembra directa 

Text :  Electronic, Optical, and Magnetic Properties of Materials
['Electronic', ',', 'Optical', ',', 'and', 'Magnetic', 'Properties', 'of', 'Materials']
Filtered Text :  electronic  optical  magnetic properties materials 

Text :  La España del Quijote
['La', 'España', 'del', 'Quijote']
Filtered Text :  la españa del quijote 

Text :  Risk Management for Projects
['Risk', 'Management', 'for', 'Projects']
Filtered Text :  risk management projects 

Text :  Smithsonian’s Objects That Define America
['Smithsonian', '’', 's', 'Objects', 'That', 'Define', 'America']
Filtered Text :  smithsonian  objects define america 

Te

Text :  Advanced Linear Algebra: Foundations to Frontiers
['Advanced', 'Linear', 'Algebra', ':', 'Foundations', 'to', 'Frontiers']
Filtered Text :  advanced linear algebra  foundations frontiers 

Text :  Entrepreneurial Opportunities
['Entrepreneurial', 'Opportunities']
Filtered Text :  entrepreneurial opportunities 

Text :  Agua en América Latina: Abundancia en medio de la escasez mundial
['Agua', 'en', 'América', 'Latina', ':', 'Abundancia', 'en', 'medio', 'de', 'la', 'escasez', 'mundial']
Filtered Text :  agua en américa latina  abundancia en medio de la escasez mundial 

Text :  Understanding Political Concepts
['Understanding', 'Political', 'Concepts']
Filtered Text :  understanding political concepts 

Text :  How to Code: Complex Data
['How', 'to', 'Code', ':', 'Complex', 'Data']
Filtered Text :  code  complex data 

Text :  Hypersonics – from Shock Waves to Scramjets
['Hypersonics', '–', 'from', 'Shock', 'Waves', 'to', 'Scramjets']
Filtered Text :  hypersonics  shock waves sc

Filtered Text :  scripting programming foundations 

Text :  Using GPUs to Scale and Speed-up Deep Learning
['Using', 'GPUs', 'to', 'Scale', 'and', 'Speed', '-', 'up', 'Deep', 'Learning']
Filtered Text :  using gpus scale speed  deep learning 

Text :  StudioX: Creating a Course with edX Studio
['StudioX', ':', 'Creating', 'a', 'Course', 'with', 'edX', 'Studio']
Filtered Text :  studiox  creating course edx studio 

Text :  The Power of Data
['The', 'Power', 'of', 'Data']
Filtered Text :  power data 

Text :  Introducción a las ondas gravitacionales
['Introducción', 'a', 'las', 'ondas', 'gravitacionales']
Filtered Text :  introducción las ondas gravitacionales 

Text :  Drones and Autonomous Systems 2: Applications in Emergency Management
['Drones', 'and', 'Autonomous', 'Systems', '2', ':', 'Applications', 'in', 'Emergency', 'Management']
Filtered Text :  drones autonomous systems   applications emergency management 

Text :  MathTrackX: Differential Calculus
['MathTrackX', ':', 'Diffe

In [10]:
# Commonly occured word in titles
Counter(lst).most_common(20)
# Counter(lst)

[('de', 97),
 ('introduction', 73),
 ('data', 56),
 ('la', 56),
 ('science', 47),
 ('introducción', 41),
 ('management', 39),
 ('en', 31),
 ('part', 28),
 ('business', 28),
 ('para', 28),
 ('learning', 27),
 ('fundamentals', 26),
 ('design', 26),
 ('python', 25),
 ('programming', 23),
 ('development', 22),
 ('sustainable', 21),
 ('marketing', 20),
 ('english', 20)]

In [11]:
df_edx[df_edx['title'].str.contains("introduction")].title.head()  # Noisy word

2                   cs  introduction computer science
5                     introduction project management
15                               introduction fintech
20    cs  introduction artificial intelligence python
21                  cs  introduction game development
Name: title, dtype: object

In [12]:
df_edx[df_edx['title'].str.contains("development")].title.head()  # Noisy word

21                     cs  introduction game development
194    poverty prosperity  understanding economic dev...
253    best start life  early childhood development s...
257                       project management development
315                introduction developmental psychology
Name: title, dtype: object

In [13]:
df_edx[df_edx['title'].str.contains("python")].title.head()  # Important word

1      programming everybody  getting started python 
17                         python basics data science
20    cs  introduction artificial intelligence python
23              cs  web programming python javascript
33                              using python research
Name: title, dtype: object

In [14]:
df_edx[df_edx['title'].str.contains("sustainable")].title.head()  # Important word

253    best start life  early childhood development s...
259    circular fashion  design  science value sustai...
314                                   sustainable cities
365                   work employment sustainable future
496                        sustainable urban development
Name: title, dtype: object

In [15]:
df_edx[df_edx['title'].str.contains("learning")].title.head()   # Important word

27                        data science  machine learning
49                                      leaders learning
52     introduction data wise  collaborative process ...
88       machine learning python  practical introduction
106    machine learning python  linear models deep le...
Name: title, dtype: object

In [16]:
df_edx[df_edx['title'].str.contains("data")].title.head()   # Important word

16            data science  r basics
17        python basics data science
27    data science  machine learning
46       data science  visualization
51         data science  probability
Name: title, dtype: object

In [17]:
df_edx[df_edx['title'].str.contains("fundamentals")].title.head()   # Noisy word

25    fundamentals neuroscience  part   electrical p...
69    computing python  fundamentals procedural prog...
77    fundamentals neuroscience  part   neurons netw...
80                                html css fundamentals
92              fundamentals neuroscience  part   brain
Name: title, dtype: object

In [18]:
df_edx[df_edx['title'].str.contains("part")].title.head()   # Noisy word

25     fundamentals neuroscience  part   electrical p...
56                         steps japanese beginners part
77     fundamentals neuroscience  part   neurons netw...
92               fundamentals neuroscience  part   brain
167                   algorithms  design analysis  part 
Name: title, dtype: object

In [19]:
#Removing unwanted words in titles
for index in range(count):
    text = df_edx['title'].iloc[index]
    filtered_text = " ".join(j for j in text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")
    df_edx['title'].iloc[index] = filtered_text

In [20]:
df_edx['title'].head(10)

0                                         learn online
1         programming everybody getting started python
2                                  cs computer science
3                                       analytics edge
4    marketing analytics marketing measurement stra...
5                                   project management
6                        leading high performing teams
7                           foundations happiness work
8       managing study stress mental health university
9                             six sigma define measure
Name: title, dtype: object

In [21]:
df_edx['institution'].describe()

count                    975
unique                   111
top       Harvard University
freq                     104
Name: institution, dtype: object

In [22]:
institutions = set(np.array(df_edx['institution']))
institutions      # Set of Institutions of all courses in EdX

{'Amazon Web Services',
 'Amnesty International',
 'Arizona State University',
 'Arm Education',
 'Australian National University',
 'Babson College',
 'Berklee College of Music',
 'Boston University',
 'Catalyst',
 'Chalmers University of Technology',
 'Columbia University',
 'Cornell University',
 'Curtin University',
 'Dartmouth College',
 'Dartmouth_IMTx-Dartmouth College-IMT',
 'Davidson College',
 'Davidson Next',
 'Delft University of Technology',
 'Delft University of Technology-Wageningen University & Research-Delft University & Wageningen University',
 'DelftXRWTHx-Delft University of Technology-RWTH Aachen University',
 'Doane University',
 'ETH Zurich',
 'Educational Testing Service',
 'Georgetown University',
 'HEC Montréal',
 'Harvard University',
 'IBM',
 'IITBombay',
 'IMT',
 'Imperial College Business School-Imperial College London',
 'Imperial College London',
 'Indian Institute of Management Bangalore',
 'Inter-American Development Bank',
 'IsraelX',
 'IsraelX-Tel Av

In [23]:
df_edx['Level'].describe()

count              975
unique               3
top       Introductory
freq               621
Name: Level, dtype: object

In [24]:
levels = set(np.array(df_edx['Level']))
levels

{'Advanced', 'Intermediate', 'Introductory'}

In [25]:
df_edx['n_enrolled'].describe()

count        855
unique       849
top       97,291
freq           2
Name: n_enrolled, dtype: object

In [26]:
# Converting n_enrolled feature into int type
for i in range(len(df_edx)):
    df_edx['n_enrolled'].iloc[i] = "".join(j for j in str(df_edx['n_enrolled'].iloc[i]) if j.isnumeric())
    if df_edx['n_enrolled'].iloc[i] != '':
        df_edx['n_enrolled'].iloc[i] = int(df_edx['n_enrolled'].iloc[i])

### Handling Null values

In [27]:
# Filling empty values with NaN values
df_edx['n_enrolled'].loc[df_edx['n_enrolled'] == ''] = np.nan

In [28]:
# Replacing NaN values with O
df_edx['n_enrolled'].fillna(0, inplace = True)

In [29]:
# Count of n_enrolled
df_edx['n_enrolled'].count()

975

In [30]:
df_edx.head()

,title,institution,Level,n_enrolled
0,learn online,edX,Introductory,124980
1,programming everybody getting started python,The University of Michigan,Introductory,293864
2,cs computer science,Harvard University,Introductory,2442271
3,analytics edge,Massachusetts Institute of Technology,Intermediate,129555
4,marketing analytics marketing measurement stra...,"University of California, Berkeley",Introductory,81140


In [31]:
# from sklearn.feature_extraction.text import CountVectorizer
# title_vectorizer = CountVectorizer()
# title_features   = title_vectorizer.fit_transform(df_edx['title'].iloc[:5])
# # title_features.get_shape()
# print(title_vectorizer.vocabulary_)
# title_features.toarray()

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(df_edx['title'])
# title_features.get_shape()
# print(title_vectorizer.vocabulary_)
print(df_edx['title'].head())
title_features.toarray()

0                                         learn online
1         programming everybody getting started python
2                                  cs computer science
3                                       analytics edge
4    marketing analytics marketing measurement stra...
Name: title, dtype: object


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Bag of Words Model
## Using Euclidean Similarity

In [33]:
# We have worked Based on Eucledean Distances but found it's less accuarate when compared to Cosine Similarity


def features(doc):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words('english')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    
    doc_feature = title_vectorizer.transform([filtered_text])
    return doc_feature

def bag_of_words_model(doc):
    doc_feature = features(doc)
    pairwise_dist = pairwise_distances(title_features, doc_feature)
    print(pairwise_dist)
    indices = np.argsort(pairwise_dist.flatten())[0:10]
    pdists  = np.sort(pairwise_dist.flatten())[0:10]
    print(df_edx.iloc[df_edx.index[indices]])
    
bag_of_words_model("Introduction to marketing")


[[1.73205081]
 [2.44948974]
 [2.        ]
 [1.73205081]
 [2.        ]
 [1.73205081]
 [2.23606798]
 [2.        ]
 [2.64575131]
 [2.23606798]
 [2.23606798]
 [2.82842712]
 [1.73205081]
 [2.        ]
 [1.73205081]
 [1.41421356]
 [2.        ]
 [2.23606798]
 [1.73205081]
 [1.73205081]
 [2.23606798]
 [2.        ]
 [2.64575131]
 [2.44948974]
 [2.82842712]
 [2.23606798]
 [1.41421356]
 [2.23606798]
 [2.64575131]
 [2.23606798]
 [1.73205081]
 [2.44948974]
 [1.73205081]
 [2.        ]
 [1.73205081]
 [2.23606798]
 [2.23606798]
 [2.23606798]
 [1.41421356]
 [3.31662479]
 [2.64575131]
 [2.23606798]
 [2.        ]
 [2.44948974]
 [2.64575131]
 [1.73205081]
 [2.        ]
 [1.73205081]
 [1.73205081]
 [1.73205081]
 [2.23606798]
 [2.        ]
 [2.82842712]
 [3.        ]
 [2.        ]
 [2.23606798]
 [2.        ]
 [1.73205081]
 [2.23606798]
 [1.41421356]
 [2.        ]
 [2.23606798]
 [2.44948974]
 [1.73205081]
 [1.41421356]
 [2.82842712]
 [2.        ]
 [2.        ]
 [1.73205081]
 [2.23606798]
 [2.        ]
 [1.73

## Using Cosine Similarity

In [34]:
def features(doc):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words('english')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                              and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    
    doc_feature = title_vectorizer.transform([filtered_text])
    print(doc_feature.toarray())
    return doc_feature

def bag_of_words_model(doc):
    doc_feature = features(doc)
    pairwise_dist = cosine_similarity(title_features, doc_feature)
    print(pairwise_dist)
    indeces = np.argsort(pairwise_dist.flatten())[-10:]
    pdists  = np.sort(pairwise_dist.flatten())[-10]
    indeces = list(indeces)
    indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    print(df_edx.iloc[df_edx.index[np.array(indeces)]])

bag_of_words_model("Introduction to python")


[[0 0 0 ... 0 0 0]]
[[0.        ]
 [0.4472136 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.5       ]
 [0.        ]
 [0.        ]
 [0.5       ]
 [0.        ]
 [0.        ]
 [0.4472136 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.57735027]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.5       ]


# TF-IDF Model
## Using Cosine Similarity

In [35]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features = tfidf_title_vectorizer.fit_transform(df_edx['title'])
tfidf_title_features.get_shape()
print(tfidf_title_vectorizer.vocabulary_)
tfidf_title_features.toarray()

{'learn': 920, 'online': 1133, 'programming': 1282, 'everybody': 589, 'getting': 704, 'started': 1507, 'python': 1315, 'cs': 367, 'computer': 295, 'science': 1415, 'analytics': 55, 'edge': 482, 'marketing': 978, 'measurement': 999, 'strategy': 1519, 'project': 1284, 'management': 966, 'leading': 918, 'high': 759, 'performing': 1185, 'teams': 1565, 'foundations': 665, 'happiness': 746, 'work': 1715, 'managing': 969, 'study': 1533, 'stress': 1524, 'mental': 1019, 'health': 753, 'university': 1648, 'six': 1463, 'sigma': 1452, 'define': 401, 'measure': 998, 'fat': 627, 'chance': 218, 'probability': 1260, 'ground': 736, 'chasing': 220, 'dream': 459, 'end': 537, 'procrastination': 1267, 'get': 703, 'job': 882, 'love': 950, 'bitcoin': 160, 'cryptocurrencies': 365, 'english': 546, 'grammar': 722, 'style': 1536, 'ethics': 584, 'action': 10, 'fintech': 643, 'data': 386, 'basics': 124, 'beer': 130, 'strategic': 1517, 'social': 1472, 'media': 1005, 'artificial': 95, 'intelligence': 846, 'game': 68

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
# TF-IDF cosine similarity model also Worked well but it is litle bit biased towards the new and rare terms in titles
"""
def features(doc):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words('english')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                              and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    
    doc_feature = tfidf_title_vectorizer.transform([filtered_text])
    return doc_feature

def bag_of_words_model(doc):
    doc_feature = features(doc)
    pairwise_dist = cosine_similarity(title_features, doc_feature)
    indeces = np.argsort(pairwise_dist.flatten())[-10:]
    pdists  = np.sort(pairwise_dist.flatten())[-10]
    #indeces = list(indeces)
    #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    print(df_edx.iloc[df_edx.index[np.array(indeces)]])

bag_of_words_model("Introduction to Python")
"""

'\ndef features(doc):\n    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words(\'english\')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")\n    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"                              and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    \n    doc_feature = tfidf_title_vectorizer.transform([filtered_text])\n    return doc_feature\n\ndef bag_of_words_model(doc):\n    doc_feature = features(doc)\n    pairwise_dist = cosine_similarity(title_features, doc_feature)\n    indeces = np.argsort(pairwise_dist.flatten())[-10:]\n    pdists  = np.sort(pairwise_dist.flatten())[-10]\n    #indeces = list(indeces)\n    #indeces.sort(key = lambda x : len(df_edx[\'title\'].iloc[x]))\n    print(df_edx.iloc[df_edx.index[np.array(indeces)]])\n\nbag_of_words_model("Introduction to Python")\n'

## Using Euclidean Similarity

In [37]:
# TF-IDF Euclidean similarity model not Worked well when compared to Cosine similarity

"""
def features(doc):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words('english')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    
    doc_feature = tfidf_title_vectorizer.transform([filtered_text])
    return doc_feature

def bag_of_words_model(doc):
    doc_feature = features(doc)
    pairwise_dist = pairwise_distances(title_features, doc_feature)
    indices = np.argsort(pairwise_dist.flatten())[0:10]
    pdists  = np.sort(pairwise_dist.flatten())[0:10]
    print(df_edx.iloc[df_edx.index[indices]])
    
bag_of_words_model("Introduction to marketing")
"""

'\ndef features(doc):\n    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(doc) if j.lower() not in stopwords.words(\'english\')] if ((i.isalpha()) and i not in string.punctuation) or i == " ")\n    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals")    \n    doc_feature = tfidf_title_vectorizer.transform([filtered_text])\n    return doc_feature\n\ndef bag_of_words_model(doc):\n    doc_feature = features(doc)\n    pairwise_dist = pairwise_distances(title_features, doc_feature)\n    indices = np.argsort(pairwise_dist.flatten())[0:10]\n    pdists  = np.sort(pairwise_dist.flatten())[0:10]\n    print(df_edx.iloc[df_edx.index[indices]])\n    \nbag_of_words_model("Introduction to marketing")\n'

# Using Multiple features

## Weighted Bag of Words Model
## Euclidean Similarity

In [38]:
df_edx.to_pickle('pickle1')
df_edx = pd.read_pickle('pickle1')
df_edx.head()

,title,institution,Level,n_enrolled
0,learn online,edX,Introductory,124980
1,programming everybody getting started python,The University of Michigan,Introductory,293864
2,cs computer science,Harvard University,Introductory,2442271
3,analytics edge,Massachusetts Institute of Technology,Intermediate,129555
4,marketing analytics marketing measurement stra...,"University of California, Berkeley",Introductory,81140


In [39]:
for i in range(len(df_edx)):
    df_edx['institution'].iloc[i] = "".join(j for j in df_edx['institution'].iloc[i].split())
df_edx['institution'].head()

C:\Users\SANTOSH ABHINAV\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0                                   edX
1               TheUniversityofMichigan
2                     HarvardUniversity
3    MassachusettsInstituteofTechnology
4       UniversityofCalifornia,Berkeley
Name: institution, dtype: object

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
institution_vectorizer = CountVectorizer()
level_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(df_edx['title'])
institution_features = institution_vectorizer.fit_transform(df_edx['institution'])
level_features = level_vectorizer.fit_transform(df_edx['Level'])
# Weights of features (Importance in recommendation)
w1 = 1
w2 = 1
w3 = 0.5
df_features = hstack((w1*title_features, w2*institution_features, w3*level_features)).tocsr()
df_features.shape

(975, 1871)

In [41]:
def features(title, institution, level):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words('english')]\
                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") 
    title_feature = title_vectorizer.transform([filtered_text])
    institution_feature = institution_vectorizer.transform([filtered_text])
    level_feature = level_vectorizer.transform([filtered_text])
    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()
    return doc_feature

def bag_of_words_model(title, institution, level):
    doc_feature = features(title, institution, level)
    pairwise_dist = pairwise_distances(df_features, doc_feature)
    indeces = np.argsort(pairwise_dist.flatten())[:10]
    pdists  = np.sort(pairwise_dist.flatten())[:10]
    #indeces = list(indeces)
    #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    print(df_edx.iloc[df_edx.index[np.array(indeces)]])
    
bag_of_words_model("Introduction to Data Science", "Harvard University", "Introductory")

                          title           institution         Level  \
107                data science                   IBM  Introductory   
46   data science visualization     HarvardUniversity  Introductory   
51     data science probability     HarvardUniversity  Introductory   
469         data science method                   IBM  Introductory   
117            sql data science                   IBM  Introductory   
860    programming data science  UniversityofAdelaide  Introductory   
233          data science tools                   IBM  Introductory   
84       data science wrangling     HarvardUniversity  Introductory   
75        data science capstone     HarvardUniversity  Introductory   
16        data science r basics     HarvardUniversity  Introductory   

     n_enrolled  
107       43766  
46       171258  
51       135859  
469        7868  
117       39184  
860       41268  
233       15627  
84        64584  
75        62803  
16       528782  


In [42]:
# from sklearn.feature_extraction.text import CountVectorizer
# print(df_edx.head())
# title_vectorizer = CountVectorizer()
# institution_vectorizer = CountVectorizer()
# level_vectorizer = CountVectorizer()
# title_features   = title_vectorizer.fit_transform(df_edx['title'].head())
# institution_features = institution_vectorizer.fit_transform(df_edx['institution'].head())
# level_features = level_vectorizer.fit_transform(df_edx['Level'].head())
# print(title_vectorizer.vocabulary_)
# print(institution_vectorizer.vocabulary_)
# print(level_vectorizer.vocabulary_)
# w1 = 1
# w2 = 0.5
# w3 = 1
# df_features = hstack((w1*title_features, w2*institution_features, w3*level_features)).tocsr()
# df_features.toarray()

In [43]:
# def features(title, institution, level):
#     filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words('english')]\
#                              if ((i.isalpha()) and i not in string.punctuation) or i == " ")
#     filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
#                              and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") 
#     title_feature = title_vectorizer.transform([filtered_text])
#     institution_feature = institution_vectorizer.transform(["".join(i for i in institution.split())])
#     level_feature = level_vectorizer.transform([level])
#     doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()
#     print(doc_feature.shape)
#     return doc_feature

# def bag_of_words_model(title, institution, level):
#     doc_feature = features(title, institution, level)
#     pairwise_dist = cosine_similarity(df_features, doc_feature)
#     print(list(enumerate(pairwise_dist)))
#     indeces = np.argsort(pairwise_dist.flatten())[-2:]
#     pdists  = np.sort(pairwise_dist.flatten())[-2:]
#     #indeces = list(indeces)
#     #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
#     df = pd.read_csv("edX.csv")
#     print(df_edx.iloc[indeces])
    
# bag_of_words_model("Analytics", "UniversityofCalifornia,Berkeley", "Introductory")

## Weighted Bag of Words Model

## Cosine Similarity

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
institution_vectorizer = CountVectorizer()
level_vectorizer = CountVectorizer()
title_features   = title_vectorizer.fit_transform(df_edx['title'])
institution_features = institution_vectorizer.fit_transform(df_edx['institution'])
level_features = level_vectorizer.fit_transform(df_edx['Level'])
w1 = 1
w2 = 1
w3 = 2
df_features = hstack((w1*title_features, w2*institution_features, w3*level_features)).tocsr()
df_features.shape

(975, 1871)

In [45]:
def features(title, institution, level):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words('english')]\
                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") 
    title_feature = title_vectorizer.transform([filtered_text])
    institution_feature = institution_vectorizer.transform(["".join(i for i in institution.split())])
    level_feature = level_vectorizer.transform([level])
    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()
    print(doc_feature.shape)
    return doc_feature

def bag_of_words_model(title, institution, level):
    doc_feature = features(title, institution, level)
    pairwise_dist = cosine_similarity(df_features, doc_feature)
    indeces = np.argsort(pairwise_dist.flatten())[-10:]
    pdists  = np.sort(pairwise_dist.flatten())[-10]
    #indeces = list(indeces)
    #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    df = pd.read_csv("edX.csv")
    print(df.iloc[indeces])
    
bag_of_words_model("Introduction to Data Science", "Harvard University", "Introductory")

(1, 1871)
                                    title  \
107          Introduction to Data Science   
79        Data Science: Linear Regression   
81       Data Science: Productivity Tools   
27         Data Science: Machine Learning   
61   Data Science: Inference and Modeling   
16                 Data Science: R Basics   
75                 Data Science: Capstone   
84                Data Science: Wrangling   
46            Data Science: Visualization   
51              Data Science: Probability   

                                               summary n_enrolled  \
107  Learn about the world of data science first-ha...     43,766   
79   Learn how to use R to implement linear regress...     75,310   
81   Keep your projects organized and produce repro...     66,683   
27   Build a movie recommendation system and learn ...   2,76,209   
61   Learn inference and modeling, two of the most ...     88,256   
16   Build a foundation in R and learn how to wrang...   5,28,782   
75   Show w

## Weighted TF-IDF Model
## Euclidean Similarity

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_institution_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_level_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features   = tfidf_title_vectorizer.fit_transform(df_edx['title'])
tfidf_institution_features = tfidf_institution_vectorizer.fit_transform(df_edx['institution'])
tfidf_level_features = tfidf_level_vectorizer.fit_transform(df_edx['Level'])
w1 = 1
w2 = 0.5
w3 = 0.5
df_features = hstack((w1*tfidf_title_features, w2*tfidf_institution_features, w3*tfidf_level_features)).tocsr()

In [47]:
# TF-IDF Euclidean similarity model Worked well but litle bised towards the new and rare Institutions

"""
def features(title, institution, level):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words('english')]\
                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") 
    title_feature = title_vectorizer.transform([filtered_text])
    institution_feature = institution_vectorizer.transform([filtered_text])
    level_feature = level_vectorizer.transform([filtered_text])
    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()
    return doc_feature

def bag_of_words_model(title, institution, level):
    doc_feature = features(title, institution, level)
    pairwise_dist = pairwise_distances(df_features, doc_feature)
    indeces = np.argsort(pairwise_dist.flatten())[:10]
    pdists  = np.sort(pairwise_dist.flatten())[:10]
    #indeces = list(indeces)
    #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    print(df_edx.iloc[df_edx.index[np.array(indeces)]])
    
bag_of_words_model("Introduction to Data Science", "Harvard University", "Introductory")
"""

'\ndef features(title, institution, level):\n    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words(\'english\')]                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")\n    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") \n    title_feature = title_vectorizer.transform([filtered_text])\n    institution_feature = institution_vectorizer.transform([filtered_text])\n    level_feature = level_vectorizer.transform([filtered_text])\n    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()\n    return doc_feature\n\ndef bag_of_words_model(title, institution, level):\n    doc_feature = features(title, institution, level)\n    pairwise_dist = pairwise_distances(df_features, doc_feature)\n    indeces = np

## Weighted TF-IDF Model

## Cosine Similarity

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_institution_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_level_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_title_features   = tfidf_title_vectorizer.fit_transform(df_edx['title'])
tfidf_institution_features = tfidf_institution_vectorizer.fit_transform(df_edx['institution'])
tfidf_level_features = tfidf_level_vectorizer.fit_transform(df_edx['Level'])
w1 = 1
w2 = 0.5
w3 = 1
df_features = hstack((w1*tfidf_title_features, w2*tfidf_institution_features, w3*tfidf_level_features)).tocsr()

In [49]:
# TF-IDF Cosine_Similarity similarity model Worked well when compared to TF-IDf Euclidean_Similarity model

"""
def features(title, institution, level):
    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words('english')]\
                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")
    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"\
                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") 
    title_feature = title_vectorizer.transform([filtered_text])
    institution_feature = institution_vectorizer.transform([filtered_text])
    level_feature = level_vectorizer.transform([filtered_text])
    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()
    return doc_feature

def bag_of_words_model(title, institution, level):
    doc_feature = features(title, institution, level)
    pairwise_dist = cosine_similarity(df_features, doc_feature)
    indeces = np.argsort(pairwise_dist.flatten())[-10:]
    pdists  = np.sort(pairwise_dist.flatten())[-10]
    #indeces = list(indeces)
    #indeces.sort(key = lambda x : len(df_edx['title'].iloc[x]))
    print(df_edx.iloc[df_edx.index[np.array(indeces)]])
    
bag_of_words_model("Introduction to Data Science", "Harvard University", "Introductory")
"""

'\ndef features(title, institution, level):\n    filtered_text = " ".join(i for i in [j.lower() for j in wordpunct_tokenize(title) if j.lower() not in stopwords.words(\'english\')]                             if ((i.isalpha()) and i not in string.punctuation) or i == " ")\n    filtered_text = " ".join(j for j in filtered_text.split() if j != "introduction" and j != "de"                             and j != "introducción" and j != "en" and j != "part" and j != "fundamentals") \n    title_feature = title_vectorizer.transform([filtered_text])\n    institution_feature = institution_vectorizer.transform([filtered_text])\n    level_feature = level_vectorizer.transform([filtered_text])\n    doc_feature = hstack((w1*title_feature, w2*institution_feature, w3*level_feature)).tocsr()\n    return doc_feature\n\ndef bag_of_words_model(title, institution, level):\n    doc_feature = features(title, institution, level)\n    pairwise_dist = cosine_similarity(df_features, doc_feature)\n    indeces = np.

In [50]:
with open("intents.json") as file:
    bot_data = json.load(file)
bot_data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Hi',
    'Hii',
    'Hai'],
   'responses': ['Hi there!, I am your course recommendation bot! I can guide you to choose the most recommended courses in edX Enter the name of the course or domain of the course you are looking for!!']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Good bye',
    'Nice chatting to you, bye',
    'Till next time',
    'Nice to meet you'],
   'responses': ['See you!',
    'Have a nice day',
    'Bye! Come back again soon.',
    'Bye! Have a great day']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    'Glad to help you!']},
  {'tag': 'no_answer',
   'patterns': [''],
   'responses': ["Sorry, can't 

In [51]:
stemmer = LancasterStemmer()
ques = []
tag_ = []
for index, intent in enumerate(bot_data['intents']):
    for i in intent['patterns']:
        ques.append(" ".join(stemmer.stem(w) for w in wordpunct_tokenize(i) if w.isalpha()))
        out = [0]*len(bot_data['intents'])
        out[index] = 1
        tag_.append(out)
print(ques)
np.save("ques.npy", ques)

['hi ther', 'how ar you', 'is anyon ther', 'hey', 'hol', 'hello', 'good day', 'hi', 'hii', 'hai', 'bye', 'see you lat', 'goodby', 'good bye', 'nic chat to you bye', 'til next tim', 'nic to meet you', 'thank', 'thank you', 'that s help', 'awesom thank', 'thank for help me', '', 'how you could help me', 'what you can do', 'what help you provid', 'how you can be help', 'what support is off', 'what do u do', 'what s yo work', 'which org thes cours ar from', 'which onlin platform the cours you ar recommend from', 'which websit thes cours ar from']


In [52]:
tag_ = np.array(tag_)
print(tag_)

[[1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]]


In [53]:
ques_vectorizer = CountVectorizer()
ques_features   = ques_vectorizer.fit_transform(ques).toarray()
ques_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [54]:
# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('accuracy')>0.95):
#             print("\nReached 95% accuracy so cancelling training!")
#             self.model.stop_training = True
# callbacks = myCallback()

# model = tf.keras.Sequential([tf.keras.layers.Dense(units = 1, input_shape = (None, ques_features.shape[1])),
#                            #tf.keras.layers.Dense(264, activation="relu", name="layer1"),
#                            #tf.keras.layers.Dense(128, activation="relu", name="layer2"),
#                            tf.keras.layers.Dense(128, activation="relu", name="layer3"),
#                            tf.keras.layers.Dense(64, activation="relu", name="layer4"),
#                            tf.keras.layers.Dense(6, activation="softmax", name="layer5")])
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# model.fit(ques_features, tag_, epochs = 100, batch_size = 9, callbacks = [callbacks])

In [55]:
# model.save('./model1')

In [56]:
# inp = "bye"
# k = " ".join(stemmer.stem(w) for w in wordpunct_tokenize(inp) if w.isalpha())
# t = ques_vectorizer.transform([k]).toarray()
# print(t)
# y = model.predict([t])
# print(y)
# ind = y.argmax()
# ind

In [57]:
# out = random.choice(bot_data['intents'][ind]['responses'])
# out

In [58]:
new_model = tf.keras.models.load_model('./model')

In [59]:
inp = "bye"
k = " ".join(stemmer.stem(w) for w in wordpunct_tokenize(inp) if w.isalpha())
t = ques_vectorizer.transform([k]).toarray()
print(t)
y = new_model.predict([t])
print(y)
ind = y.argmax()
ind

[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0.04585503 0.80303407 0.0541085  0.01716744 0.01647774 0.0633572 ]]


1

In [60]:
out = random.choice(bot_data['intents'][ind]['responses'])
out

'Have a nice day'